## Algorithm benchmarking
- This file handles the timing of execution time for the different alogrithms in different scenarios(best,worst,average).
- The results are stored in csv files for later visualization.
- This file also contains the actual algorithms that are being analyzed.

In [ ]:
#Necessary imports
import timeit
import copy
import pandas as pd

In [ ]:
#The sorting algorithms

#Insertion sort, based on the pseudocode from lecture 7, 26/02/2024
def insert_sort(A):
    for j in range(1, len(A)):
        key = A[j]
        i = j - 1
        while i >= 0 and A[i] > key:
            A[i + 1] = A[i]
            i = i - 1
        A[i+ 1] = key
    return A

# Merge sort, psudo code from lecture 7, 26/02/2024
def merge(A, p, q, r):
    n1 = q - p + 1
    n2 = r - q

    L = [0] * (n1+1)
    R = [0] * (n2+1)

    for i in range(n1):
        L[i] = A[p + i]
    
    for j in range(n2):
        R[j] = A[q + j + 1]

    L[n1] = float('inf')
    R[n2] = float('inf')

    i = 0
    j = 0

    for k in range(p, r+1):
        if L[i] <= R[j]:
            A[k] = L[i]
            i = i + 1
        else:
            A[k] = R[j]
            j = j + 1
    return A

# A  is the array to be sorted
# p is the start index
# q is the middle index
# r is the end index
def merge_sort(A, p, r):
    if p < r:
        q = (p + r) // 2
        merge_sort(A, p, q)
        merge_sort(A, q+1, r)
        merge(A, p, q, r)
        #A =  List cointaing two lists, p = start index first list, q = middle index end first list, start sekond list, r =end second list
    return A

# Quick sort, psudo code from lecture 7, 26/02/2024
def partition(A, p, q):
    x = A[p]
    i = p
    for j in range(p+1, q+1):
        if A[j] <= x:
            i = i + 1
            A[i], A[j] = A[j], A[i]
    A[i], A[p] = A[p], A[i]
    return i 

def quick_sort(A, p, r):
    if p < r:
        q = partition(A, p, r)
        quick_sort(A, p, q - 1)
        quick_sort(A, q + 1, r)

In [ ]:
#The function to benchmark the sorting algorithms
def timing_function(list_name, list_of_lists, func_name, limit, df):
    '''
    Benchmark the sorting algorithms on a list of lists
    Using the timeit module to measure the time it takes to sort a list of a given size
    
    Parameters:
    list_name: str, the name of the list type
    list_of_lists: list, a list of lists to be sorted
    func_name: str, the name of the sorting function to be benchmarked
    limit: int, the maximum number of lists to be sorted
    df: pandas dataframe, the dataframe to store the timing results
    '''
    for i in range(0, len(list_of_lists)): #Iterate over the list of lists  
        current_list = [int(x) for x in list_of_lists[i].split(',')]
        print("Benchmarking {} with list of length {} of type {}".format(func_name, len(current_list), list_name))
        if func_name == "insert_sort":
            clock = timeit.Timer(stmt='sort_func(copy(data))', globals={'sort_func': insert_sort, 'data': current_list, 'copy': copy.copy})    
        elif func_name == "merge_sort":
            clock = timeit.Timer(stmt='sort_func(copy(data),0,r)', globals={'sort_func': merge_sort, 'data': current_list, 'copy': copy.copy, 'r': len(current_list)-1, "0": 0})
        elif func_name == "quick_sort":
            clock = timeit.Timer(stmt='sort_func(copy(data),0,r)', globals={'sort_func': quick_sort, 'data': current_list, 'copy': copy.copy, 'r': len(current_list)-1, "0": 0})
        n_ar, t_ar = clock.autorange()
        t = clock.repeat(repeat=7, number=n_ar)

        for time in t:
            new_row = {'list_type': list_name, 'list_size': len(current_list), 'n_ar': n_ar, 'time': time}
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)        #Store the timing results in a csv file
  
        #Stop the benchmarking if the limit is reached
        if i == limit-1:
            break
    return df

In [ ]:
#Read the csv files containing the benchmarking data
b_c_file = open("../data/b_c.csv", "r") #Best case
w_c_file = open("../data/w_c.csv", "r") #Worst case
a_c_file = open("../data/a_c.csv", "r") #Average case

#Read the file and store the data in a list
a_c = a_c_file.read().splitlines()
b_c = b_c_file.read().splitlines()
w_c = w_c_file.read().splitlines()

In [ ]:
#Limit the number of lists to be sorted. Must be less than or equal to the number of lists in the input files (25)
limit = 3 #Must be >= 1, and quick_sort will have problems with recursion depth if larger than 11
algorithm_list = ["insert_sort", "merge_sort", "quick_sort"]

#Benchmark the sorting algorithms. The timing results are stored in the csv files in the results folder
for algorithm in algorithm_list: #For each algorithm
    df = pd.DataFrame(columns=['list_type', 'list_size', 'n_ar', 'time'])
    #Call the timing function for each list type
    df = timing_function("a_c", a_c, algorithm,limit, df)
    df = timing_function("b_c", b_c, algorithm,limit, df)
    df = timing_function("w_c", w_c, algorithm,limit, df)

    #Store the timing results in a pickle file
    df.to_pickle(f"../data/{algorithm}_timing.pkl")